In [2]:
%load_ext autoreload
%autoreload 2
from app import db
import numpy as np
from app.models import (
    Bracket,
    Match,
    Round,
    Tournament,
    User,
    bracket_entrants 
)
from sqlalchemy.sql.expression import func, select


In [3]:
db.session.close()

In [4]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [5]:
# purge all tables first
for Model in [Bracket, Match, Round, Tournament, User]:
    purge(Model)
db.session.commit()

In [6]:
# Fall Charity LAN 2018 Melee Singles users
users = [
    User(username='TPN', email='TPN@example.com'.lower()), 
    User(username='AngryFalco', email='AngryFalco@example.com'.lower()), 
    User(username='Sunrisebanana', email='Sunrisebanana@example.com'.lower()), 
    User(username='Ptolemy', email='Ptolemy@example.com'.lower()), 
    User(username='Vik', email='Vik@example.com'.lower()), 
    User(username='Kevin', email='Kevin@example.com'.lower()), 
    User(username='Spaceghost', email='Spaceghost@example.com'.lower()), 
    User(username='Burnaby', email='Burnaby@example.com'.lower()), 
]

[db.session.add(u) for u in users]
db.session.commit()

In [7]:
users = User.query.all()
np.random.shuffle(users)

In [8]:

# set up tournament with all users
tournament_name = 'Fall Charity LAN 2018 Melee Singles'
t = \
    Tournament(
        n_entrants = len(users), 
        name=tournament_name,
        organizer_id=User.query.order_by(func.random()).first().id
    )
db.session.add(t)
db.session.commit()

In [9]:
t

<Tournament 1>

In [10]:
b = \
    Bracket(
        bracket_type='Winners',
        users=[u for u in User.query.order_by(func.random()).limit(t.n_entrants).all()],
        tournament=t,
    )

db.session.add(b)
db.session.commit()

In [11]:
b

<Winners bracket: tournament 1>

In [12]:
usrs = [u for u in User.query.order_by(func.random()).limit(t.n_entrants).all()]
# usrs = [(usrs[i], usrs[i+1]) for i in range(0, len(usrs)//2)]

In [13]:
usrs[0]

<User TPN>

In [14]:
r = \
    Round(
        number=1,
        winners=True,
        bracket=b,
#         matches=matches
    )
r

round 1 in bracket None

In [15]:
db.session.add(r)
db.session.commit()

In [16]:
r.id

1

In [17]:
matches = \
    [Match(
#         users=list((usrs[i], usrs[i+1])),
        user_1 = usrs[i].id,
        user_2 = usrs[i+1].id,
        round_id=r.id
    ) for i in range(0, len(usrs), 2)]
m = matches[0]
m

<match between 1 and 2>

In [18]:
for m in matches:
    db.session.add(m)

In [19]:
db.session.commit()

In [20]:
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)
df

,id,username,email,password_hash,about_me,last_seen
0,1,TPN,tpn@example.com,None,None,2020-02-10 22:52:28.392729
1,2,AngryFalco,angryfalco@example.com,None,None,2020-02-10 22:52:28.393427
2,3,Sunrisebanana,sunrisebanana@example.com,None,None,2020-02-10 22:52:28.393682
3,4,Ptolemy,ptolemy@example.com,None,None,2020-02-10 22:52:28.393821
4,5,Vik,vik@example.com,None,None,2020-02-10 22:52:28.393936
5,6,Kevin,kevin@example.com,None,None,2020-02-10 22:52:28.394046
6,7,Spaceghost,spaceghost@example.com,None,None,2020-02-10 22:52:28.394324
7,8,Burnaby,burnaby@example.com,None,None,2020-02-10 22:52:28.394447


In [21]:
pd.read_sql_query('select * from tournament', cnx)

,id,n_entrants,name,organizer_id
0,1,8,Fall Charity LAN 2018 Melee Singles,4


In [22]:
pd.read_sql_query('select * from bracket', cnx)

,id,bracket_type,tournament_id
0,1,Winners,1


In [23]:
pd.read_sql_query('select * from bracket_entrants', cnx)

,user_id,bracket_id
0,1,1
1,8,1
2,3,1
3,2,1
4,6,1
5,4,1
6,7,1
7,5,1


In [24]:
pd.read_sql_query('select * from round', cnx)

,id,number,winners,bracket_id
0,1,1,1,1


In [25]:
pd.read_sql_query('select * from match', cnx)

,id,user_1,user_2,round_id
0,1,1,2,1
1,2,4,7,1
2,3,8,5,1
3,4,3,6,1
